In [21]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from pytimekr import pytimekr

In [32]:
df = pd.read_csv('./data/28-28_train.csv')
test_df = pd.read_csv('./data/28_test.csv')
df.head()

,ID,timestamp,item,corporation,location,supply,price,price_t-28,price_t-27,price_t-26,...,price_t + 18,price_t + 19,price_t + 20,price_t + 21,price_t + 22,price_t + 23,price_t + 24,price_t + 25,price_t + 26,price_t + 27
0,TG_A_J_20190129,2019-01-29,TG,A,J,24969.0,2472.0,0.0,0.0,1728.0,...,0.0,2089.0,1581.0,2516.0,3730.0,3648.0,2310.0,0.0,2998.0,2249.0
1,TG_A_J_20190130,2019-01-30,TG,A,J,27852.0,2377.0,0.0,1728.0,1408.0,...,2089.0,1581.0,2516.0,3730.0,3648.0,2310.0,0.0,2998.0,2249.0,1672.0
2,TG_A_J_20190131,2019-01-31,TG,A,J,15997.0,2278.0,1728.0,1408.0,1250.0,...,1581.0,2516.0,3730.0,3648.0,2310.0,0.0,2998.0,2249.0,1672.0,3841.0
3,TG_A_J_20190201,2019-02-01,TG,A,J,36010.0,2349.0,1408.0,1250.0,0.0,...,2516.0,3730.0,3648.0,2310.0,0.0,2998.0,2249.0,1672.0,3841.0,4307.0
4,TG_A_J_20190202,2019-02-02,TG,A,J,8808.0,2286.0,1250.0,0.0,1474.0,...,3730.0,3648.0,2310.0,0.0,2998.0,2249.0,1672.0,3841.0,4307.0,4538.0


In [33]:
test_df.head()

,ID,timestamp,item,corporation,location,price_t-28,price_t-27,price_t-26,price_t-25,price_t-24,...,price_t-10,price_t-9,price_t-8,price_t-7,price_t-6,price_t-5,price_t-4,price_t-3,price_t-2,price_t-1
0,TG_A_J_20230304,2023-03-04,TG,A,J,1864.0,0.0,1837.0,1595.0,1747.0,...,2813.0,2770.0,2633.0,3155.0,0.0,2657.0,3922.0,3397.0,3195.0,3640.0
1,TG_A_S_20230304,2023-03-04,TG,A,S,2656.0,0.0,2312.0,2024.0,2874.0,...,3444.0,3481.0,3518.0,4201.0,0.0,4166.0,4009.0,4173.0,4219.0,4089.0
2,TG_B_J_20230304,2023-03-04,TG,B,J,1839.0,0.0,1410.0,1322.0,2316.0,...,4175.0,6216.0,3558.0,2412.0,0.0,3540.0,3141.0,6382.0,3558.0,3470.0
3,TG_B_S_20230304,2023-03-04,TG,B,S,1884.0,0.0,1897.0,1988.0,1711.0,...,3202.0,3478.0,3939.0,3677.0,0.0,4057.0,3821.0,4037.0,4004.0,4241.0
4,TG_C_J_20230304,2023-03-04,TG,C,J,2542.0,0.0,2366.0,2553.0,2785.0,...,4017.0,4585.0,4835.0,5550.0,0.0,5037.0,2643.0,3742.0,3983.0,5175.0


In [34]:
holiday = []
for year in range(2019, 2024):
    holiday += pytimekr.holidays(year=year)



def prerpocess(df, holiday=holiday):
    is_train = 'price(원/kg)' in df.columns
    is_holiday = []

    df.timestamp = pd.to_datetime(df.timestamp)
    df['year'] = df.timestamp.dt.year
    df['month'] = df.timestamp.dt.month
    # df['day'] = df.timestamp.dt.day
    df['dow'] = df.timestamp.dt.day_of_week

    for time, dow in zip(df.timestamp, df.dow):
        if time in holiday or dow == 6:
            is_holiday.append(1)
        else:
            is_holiday.append(0)
    df['is_holiday'] = is_holiday
    # df = pd.get_dummies(df, columns=['month'])
    
    if is_train:
        df = pd.get_dummies(df, columns=['corporation', 'location', 'dow', 'month'])
        df.drop(['ID', 'timestamp', 'supply'], axis=1, inplace=True)
        # df.rename(columns={'price(원/kg)' : 'price'}, inplace=True)
        # df = df.loc[(df['price'] != 0) | (df['dow'] != 6)]
    else:
        df = pd.get_dummies(df, columns=['corporation', 'location'])
        for i in range(1, 13):
            df[f'month_{i}'] = 0
        for i in range(7):
            df[f'dow_{i}'] = 0
        df['month_3'] = 1
        df['dow_5'] = 1
        df.drop(['ID', 'timestamp', 'month', 'dow'], axis=1, inplace=True)
    
    return df

In [35]:
prerpocess(df).to_csv('./data/train_28-28_preproc_5.csv', index=False)
prerpocess(test_df).to_csv('./data/test_28_preproc_5.csv', index=False)